In [5]:
import pandas as pd
import numpy as np

# 1. Load Data
participants = pd.read_csv("C3DC Participants Download 2025-11-12 16-32-53.csv")
diagnosis = pd.read_csv("C3DC Diagnosis Download 2025-11-12 16-33-56.csv")
treatment = pd.read_csv("C3DC Treatment Download 2025-11-12 16-34-22.csv")
survival = pd.read_csv("C3DC Survival Download 2025-11-12 16-34-47.csv")  # <--- Added Survival File

# ---------------------------------------------------------
# Step 1: Target Variable (Classification) - CNS Tumor Flag
# ---------------------------------------------------------
cns_codes = ['C70', 'C71', 'C72']

def check_cns(site):
    if pd.isna(site):
        return 0
    for code in cns_codes:
        if code in str(site):
            return 1
    return 0

diagnosis['is_cns_tumor'] = diagnosis['Anatomic Site'].apply(check_cns)
# Aggregate per participant (Max: if any diagnosis is CNS, label as 1)
target_df = diagnosis.groupby('Participant ID')['is_cns_tumor'].max().reset_index()

# ---------------------------------------------------------
# Step 2: Feature - Age (Numeric)
# ---------------------------------------------------------
diagnosis['Age_numeric'] = pd.to_numeric(diagnosis['Age at Diagnosis (days)'], errors='coerce')
# Take the minimum age (age at first diagnosis)
age_df = diagnosis.groupby('Participant ID')['Age_numeric'].min().reset_index()
age_df.rename(columns={'Age_numeric': 'Age_at_Diagnosis_Days'}, inplace=True)

# ---------------------------------------------------------
# Step 3: Feature - Treatment (One-Hot Encoding)
# ---------------------------------------------------------
treatment['val'] = 1
treatment_pivot = treatment.pivot_table(
    index='Participant ID',
    columns='Treatment Type',
    values='val',
    fill_value=0,
    aggfunc='max'
).reset_index()

# Clean column names
new_cols = ['Participant ID'] + [f'Treatment_{col.replace(" ", "_")}' for col in treatment_pivot.columns if col != 'Participant ID']
treatment_pivot.columns = new_cols

# ---------------------------------------------------------
# Step 4: Survival Data Processing (New Step!)
# ---------------------------------------------------------
# Convert Age at Last Contact to numeric
survival['Age_Last_Contact'] = pd.to_numeric(survival['Age at Last Known Survival Status'], errors='coerce')

# Define Event (1=Dead, 0=Alive/Censored)
# "Dead" is the event. "Alive", "Not Reported", "Unknown" are censored (0).
survival['Event_Status'] = survival['Last Known Survival Status'].apply(lambda x: 1 if x == 'Dead' else 0)

# Select only necessary columns to avoid duplicates
survival_clean = survival[['Participant ID', 'Age_Last_Contact', 'Event_Status']].drop_duplicates('Participant ID')

# ---------------------------------------------------------
# Step 5: Merging All Data
# ---------------------------------------------------------
df_merged = pd.merge(participants, target_df, on='Participant ID', how='left')
df_merged = pd.merge(df_merged, age_df, on='Participant ID', how='left')
df_merged = pd.merge(df_merged, treatment_pivot, on='Participant ID', how='left')
df_merged = pd.merge(df_merged, survival_clean, on='Participant ID', how='left') # <--- Merge Survival

# ---------------------------------------------------------
# Step 6: Final Preprocessing & Calculations
# ---------------------------------------------------------

# 1. Filter: Keep only rows where 'is_cns_tumor' is defined
df_merged = df_merged.dropna(subset=['is_cns_tumor'])

# 2. Impute Missing Age (Median) & Add Flag
median_age = df_merged['Age_at_Diagnosis_Days'].median()
df_merged['Age_missing_flag'] = df_merged['Age_at_Diagnosis_Days'].apply(lambda x: 1 if pd.isna(x) else 0)
df_merged['Age_at_Diagnosis_Days'] = df_merged['Age_at_Diagnosis_Days'].fillna(median_age)

# 3. Fill Missing Treatment info with 0
treat_cols = [c for c in df_merged.columns if c.startswith('Treatment_')]
df_merged[treat_cols] = df_merged[treat_cols].fillna(0).astype(int)

# 4. Handle Categorical Missings (Race, Sex)
categorical_cols = ['Race', 'Sex at Birth']
for col in categorical_cols:
    df_merged[col] = df_merged[col].fillna('Unknown')
    df_merged[col] = df_merged[col].replace('Not Reported', 'Unknown')

# 5. Calculate Survival Time (Time-to-Event)
# Survival Time = Age at Last Contact - Age at Diagnosis
df_merged['Survival_Time_Days'] = df_merged['Age_Last_Contact'] - df_merged['Age_at_Diagnosis_Days']

# 6. One-Hot Encoding for Bayesian Horseshoe Prior
df_final = pd.get_dummies(df_merged, columns=categorical_cols, prefix=['Race', 'Sex'], drop_first=False)

# 7. Handle Study ID (for Hierarchical Model)
df_final['Study ID'] = df_final['Study ID'].fillna('Unknown_Study')

# ---------------------------------------------------------
# Output
# ---------------------------------------------------------
output_filename = "project_data_complete_survival.csv"
df_final.to_csv(output_filename, index=False)

print(f"Success! File created: {output_filename}")
print(f"Data Shape: {df_final.shape}")
print("-" * 30)
print("Key Columns for Analysis:")
print("1. Classification Target: 'is_cns_tumor'")
print("2. Survival Target: 'Survival_Time_Days' (Time) and 'Event_Status' (Event)")
print("3. Group Variable: 'Study ID'")

Success! File created: project_data_complete_survival.csv
Data Shape: (5056, 51)
------------------------------
Key Columns for Analysis:
1. Classification Target: 'is_cns_tumor'
2. Survival Target: 'Survival_Time_Days' (Time) and 'Event_Status' (Event)
3. Group Variable: 'Study ID'
